# Deployment of Automated Machine Learning Model

## Quick Start Overview

In this quickstart, you will start with a model that was trained using Automated Machine Learning. Learn how to use the Azure ML Python SDK to register, package, and deploy the trained model to Azure Container Instance as a scoring web service. Finally, test the deployed model (1) by make direct calls on service object, (2) by calling the service end point (Scoring URI) over http.

Because you will be using the Azure Machine Learning SDK, you will be able to provision all your required Azure resources directly from this notebook, without having to use the Azure Portal to create any resources.

Run the following cell and verify that your:
- Azure ML SDK version is 1.0.62 or later.

In [8]:
import os
import azureml
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.run import Run
from azureml.core.experiment import Experiment

from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image

# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.3.0

## Configure access to the Azure Machine Learning resources

Run the following cell to load the Azure Machine Learning service shared configuration.

In [9]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "220fc532-6091-423c-8ba0-66c2397d591b"

#Provide values for the existing Resource Group 
resource_group = "iot-lab-2020"

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "Cosmos-DB-IoT-ML-2klrbk7bxl3tk"
workspace_region = "East US"

print("Finished setting Azure Machine Learning service variables.")

Finished setting Azure Machine Learning service variables.

In [10]:
# constants, you can leave these values as they are or experiment with changing them after you have completed the notebook once
deployment_folder = './deploy'

# this is the name of the web service deployed on the AKS cluster
aci_service_name ='maint-service'

os.makedirs(deployment_folder, exist_ok=True)

## Define deployment helpers
Run the following cells to define the helper methods that will be used to perform deployment

In [11]:
def getOrCreateWorkspace(subscription_id, resource_group, workspace_name, workspace_region):
    # By using the exist_ok param, if the workspace already exists we get a reference to the existing workspace instead of an error
    ws = Workspace.create(
        name = workspace_name,
        subscription_id = subscription_id,
        resource_group = resource_group, 
        location = workspace_region,
        exist_ok = True)
    return ws

In [12]:
def deployModelAsWebService(ws, model_name, 
                scoring_script_filename="scoring_service.py", 
                conda_packages=['numpy','pandas','scikit-learn','py-xgboost<=0.80'],
                pip_packages=['azureml-train-automl==1.0.60','inference-schema'],
                conda_file="dependencies.yml", runtime="python",
                cpu_cores=1, memory_gb=1, tags={'name':'scoring'},
                description='Forecast',
                service_name = "scoring"
               ):
    # retrieve a reference to the already registered model
    print("Retrieving model reference...")
    registered_model = Model(workspace=ws, name=model_name)

    # create a Conda dependencies environment file
    print("Creating conda dependencies file locally...")
    from azureml.core.conda_dependencies import CondaDependencies 
    mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
    with open(conda_file,"w") as f:
        f.write(mycondaenv.serialize_to_string())
        
    # create container image configuration
    print("Creating container image configuration...")
    from azureml.core.image import ContainerImage
    image_config = ContainerImage.image_configuration(execution_script = scoring_script_filename,
                                                      runtime = runtime,
                                                      conda_file = conda_file)
    
    # create ACI configuration
    print("Creating ACI configuration...")
    from azureml.core.webservice import AciWebservice, Webservice
    aci_config = AciWebservice.deploy_configuration(
        cpu_cores = cpu_cores, 
        memory_gb = memory_gb, 
        tags = tags, 
        description = description)

    # deploy the webservice to ACI
    print("Deploying webservice to ACI...")
    webservice = Webservice.deploy_from_model(
      workspace=ws, 
      name=service_name, 
      deployment_config=aci_config,
      models = [registered_model], 
      image_config=image_config
    )
    webservice.wait_for_deployment(show_output=True)
    
    return webservice

## Create the scoring web service script
Run the following cell to generate the web service script that will be deployed and uses the model for prediction.

In [13]:
#save script to  $deployment_folder/scoring_service.py
scoring_service = """
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

# from inference_schema.schema_decorators import input_schema, output_schema
# from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
# from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame({'battery_Age_Days': 120, 'daily_Trip_Duration': 590})
### input_sample = pd.DataFrame(data=[{"Date":"2013-01-01T00:00:00.000Z","Battery_ID":0,"Battery_Age_Days":0,"Daily_Trip_Duration":67.8456075842}])

def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'batt-cycles-7')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
def run(data):
    try:
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

    return json.dumps({"result": result.tolist()})
"""

with open("scoring_service.py", "w") as file:
  file.write(scoring_service)

1174

Run the following to get a reference to your Azure Machine Learning workspace and retrieve the model.

In [14]:
ws =  getOrCreateWorkspace(subscription_id, resource_group, 
                   workspace_name, workspace_region)

registredModel = Model(workspace = ws, name = "batt-cycles-7")
registredModel

Model(workspace=Workspace.create(name='Cosmos-DB-IoT-ML-2klrbk7bxl3tk', subscription_id='220fc532-6091-423c-8ba0-66c2397d591b', resource_group='iot-lab-2020'), name=batt-cycles-7, id=batt-cycles-7:3, version=3, tags={}, properties={})

Run the following to deploy the web service. This will take about **5-7 minutes**.

In [ ]:
webservice = deployModelAsWebService(ws, model_name="batt-cycles-7", service_name = aci_service_name)

You have completed this notebook and can return to the lab guide.